In [1]:
# 1. Install the required packages
# On Windows, you just need to execute this cell for once.
try:
    import google.colab
    # IN_COLAB = True
except ImportError:
    # IN_COLAB = False
    %pip install -q git+https://github.com/huggingface/transformers
    %pip install -q git+https://github.com/huggingface/accelerate

%pip install -q git+https://github.com/huggingface/diffusers
%pip install -q gradio ftfy tensorboard
%pip install -q bitsandbytes
#%pip install -U git+https://github.com/TimDettmers/bitsandbytes.git
%pip install -q xformers --index-url https://download.pytorch.org/whl/cu124
#%pip install -U git+https://github.com/facebookresearch/xformers.git@main
print("Package installation finished.")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.5 MB/s eta 0:00:

In [3]:
# 2. Create folders and download training scripts
import os, shutil

dataset_dir = "./dataset"
output_dir = "./output"
logging_dir = "./logs"

# Create the directories if they don't exist
os.makedirs(dataset_dir, exist_ok=True)
# Delete the 'output' folder and its contents
shutil.rmtree(output_dir, ignore_errors=True)
os.makedirs(output_dir, exist_ok=True)
# Delete the 'log' folder and its contents
shutil.rmtree(logging_dir, ignore_errors=True)
os.makedirs(logging_dir, exist_ok=True)
# Delete the 'dataset' folder and its contents
# shutil.rmtree(dataset_dir, ignore_errors=True)
# os.makedirs(dataset_dir, exist_ok=True)

# fetch textual inversion code if it doesn't exist
if not os.path.exists("textual_inversion.py"):
    !wget https://raw.githubusercontent.com/jomo0825/MrFuGenerativeAI/main/TextualInversion/textual_inversion.py
else:
    print("textual_inversion.py already exists, skipping download.")

ipynb_checkpoints = os.path.join( dataset_dir, ".ipynb_checkpoints")
shutil.rmtree(".gradio", ignore_errors=True)
shutil.rmtree(".config", ignore_errors=True)
shutil.rmtree(ipynb_checkpoints, ignore_errors=True)

textual_inversion.py already exists, skipping download.


In [3]:
# 3. Upload dataset images
def load_dataset():
  try:
    from google.colab import files
    import os

    # Upload files from local machine
    uploaded = files.upload()

    # Move the uploaded files to the target directory
    for filename in uploaded.keys():
        # Get source and destination paths
        source_path = filename
        destination_path = os.path.join(dataset_dir, filename)

        # Move the file
        !mv "{source_path}" "{destination_path}"
        print(f"Moved {filename} to {dataset_dir}")

  except ImportError:
    import tkinter as tk
    from tkinter import filedialog
    import os

    # Initialize tkinter
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    root.attributes('-topmost',True)

    # Open a file dialog and allow multiple file selection
    file_paths = filedialog.askopenfilenames(
        title='Select Dataset Images',
        filetypes=[('Image Files', '*.png;*.jpg;*.jpeg;*.bmp;*.gif')]
    )
    root.destroy()

    # Define the target directory
    target_directory = dataset_dir

    # Copy or move files to the target directory
    for file_path in file_paths:
        filename = os.path.basename(file_path)
        destination = os.path.join(target_directory, filename)
        os.rename(file_path, destination)  # or use shutil.copy for copying
        print(f'File {filename} saved to {target_directory}')

load_dataset()

Saving sks (2).png to sks (2).png
Saving sks (3).png to sks (3).png
Saving sks (4).png to sks (4).png
Saving sks (5).png to sks (5).png
Saving sks (6).png to sks (6).png
Saving sks (1).jpg to sks (1).jpg
Saving sks (7).png to sks (7).png
Moved sks (2).png to ./dataset
Moved sks (3).png to ./dataset
Moved sks (4).png to ./dataset
Moved sks (5).png to ./dataset
Moved sks (6).png to ./dataset
Moved sks (1).jpg to ./dataset
Moved sks (7).png to ./dataset


In [4]:
import gradio as gr
from textual_inversion import main as train_textaul_inversion
from textual_inversion import parse_args
import threading, os, logging, time
from os import path
from PIL import Image

def parse_lr_schedule(lr_schedule_str):
    schedule = []
    segments = lr_schedule_str.split(',')
    for segment in segments:
        if ':' in segment:
            lr, steps = segment.split(':')
            schedule.append((float(lr), int(steps)))
        else:
            schedule.append((float(segment), None))  # Final constant learning rate
    return schedule

def get_learning_rate_at_step(lr_schedule, step):
    current_step = 0
    for lr, segment_steps in lr_schedule:
        if segment_steps is None or step < current_step + segment_steps:
            return lr
        current_step += segment_steps
    return lr_schedule[-1][0]  # Return the last LR if beyond defined steps

# Callback to update the preview image in the UI
def preview_callback(image, step):
    global current_preview, current_status, max_train_steps, current_step
    current_step = step
    current_preview = image
    # current_status = f"Preview updated at step {step}"
    # current_preview.show()
    print(f"{step}/{max_train_steps}")


def run_training(dataset_path, prompt, placeholder_token, initializer_token, num_training_steps,
                 learning_rate, batch_size, preview_save_steps, preview_seed):
    global current_preview, current_status
    global max_train_steps, current_step
    current_preview = None  # Reset the preview
    current_status = "Training started..."  # Initial status

    # Construct the command with all arguments
    command = [
        # "python", "textual_inversion.py",
        "--pretrained_model_name_or_path", "stable-diffusion-v1-5/stable-diffusion-v1-5",
        "--train_data_dir", dataset_path,
        "--placeholder_token", placeholder_token,
        "--initializer_token", initializer_token,
        "--resolution", "512",
        "--train_batch_size", str(batch_size),
        "--gradient_accumulation_steps", "1",
        "--learning_rate", str(learning_rate),
        "--max_train_steps", str(num_training_steps),
        "--save_steps", str(preview_save_steps),
        "--validation_steps", str(preview_save_steps),
        "--output_dir", output_dir,
        "--logging_dir", logging_dir,
        "--validation_prompt", prompt,
        "--learnable_property", "object",
        "--seed", str(preview_seed)
    ]

    # Print the command for debugging
    # print("Command:", " ".join(command))

    # Create the directories if they don't exist
    os.makedirs(logging_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)

    # Disable logging
    # logging.getLogger("accelerate").disabled = True

    # Run the command in a separate process
    # process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Run the command as function
    args = parse_args(command)
    max_train_steps = args.max_train_steps

    def worker(finish_event):
        try:
            train_textaul_inversion(args, {"_callback": preview_callback})
        except Exception as e:
            print(f"Training error: {e}")
        finally:
            finish_event.set()

    finish_event = threading.Event()
    finish_event.clear()
    # try:
    train_thread = threading.Thread(target=worker, args=(finish_event,))
    train_thread.start()
    yield gr.update(value=None), gr.update(value=f"Training started.")

    # except Exception as e:
    #   print(f"Error: {e}")


    while not finish_event.is_set():
        if current_preview is not None:
            yield gr.update(value=current_preview), gr.update(value=f"Preview at {current_step} step.")
            current_preview = None
        time.sleep(1)

    train_thread.join()

    # Print the output and errors (for debugging)
    # print("Output:", stdout.decode())
    # print("Errors:", stderr.decode())

    # Update status when training completes
    current_status = "Training completed!"

    yield gr.update(), gr.update(value=current_status)


def ui():
    with gr.Blocks() as demo:
        gr.Markdown("# Stable Diffusion Textual Inversion UI")
        gr.Markdown("Generate images using a preloaded textual inversion model.")

        with gr.Row():
            dataset_path = gr.Textbox(label="Dataset Path", value="dataset", interactive=True)

        with gr.Row():
            with gr.Column(scale=1, min_width=300):
                placeholder_token = gr.Textbox(label="Placeholder Token", placeholder="Enter placeholder token here", interactive=True)
                initializer_token = gr.Textbox(label="Initializer Token", placeholder="Enter initializer token here", interactive=True)
                prompt = gr.Textbox(label="Preview Prompt", placeholder="Enter your prompt here", interactive=True)
                num_training_steps = gr.Number(label="Number of Training Steps", value=100, interactive=True)
                learning_rate = gr.Number(label="Learning Rate", value=0.001, interactive=True)
                batch_size = gr.Number(label="Batch Size", value=4, interactive=True)
                preview_save_steps = gr.Number(label="Preview/Save Every N Steps", value=10, interactive=True)
                preview_seed = gr.Number(label="Preview Seed", value=1, interactive=True)
            with gr.Column(scale=1, min_width=300):
                output_image = gr.Image(label="Generated Image")
                generate_status = gr.Textbox(value="Status messages will appear here.", label="Status", interactive=False)
                generate_button = gr.Button("Start Training")


        generate_button.click(
            fn=run_training,
            inputs=[dataset_path, prompt, placeholder_token, initializer_token, num_training_steps,
                    learning_rate, batch_size, preview_save_steps, preview_seed],
            outputs=[output_image, generate_status],
            show_progress=True,
            queue=True
        )

    return demo

demo = ui()

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://15ef8fa8048a9ae254.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [16]:
demo.close()

Closing server running on port: 7865


In [34]:
import importlib
importlib.reload(textual_inversion)

<module 'textual_inversion' from '/content/textual_inversion.py'>

In [8]:
# import argparse

# Create parser
# parser = argparse.ArgumentParser(description='Your script description')
# parser.add_argument('--seed', type=int, help='Argument 1')
# parser.add_argument('--arg2', type=str, help='Argument 2')

# Create fake args list and parse directly
command2 = [
        # "python", "textual_inversion.py",
        # "--pretrained_model_name_or_path", "stable-diffusion-v1-5/stable-diffusion-v1-5",
        # "--train_data_dir", dataset_dir,
        # "--placeholder_token", placeholder_token,
        # "--initializer_token", initializer_token,
        # "--resolution", 512,
        # "--train_batch_size", batch_size,
        # "--gradient_accumulation_steps", 1,
        # "--learning_rate", learning_rate,
        # "--max_train_steps", num_training_steps,
        # "--save_steps", preview_save_steps,
        # "--validation_steps", preview_save_steps,
        # "--output_dir", output_dir,
        # "--logging_dir", logging_dir,
        # "--validation_prompt", prompt,
        # "--learnable_property", "object",
        "--seed", str(1)
]

args2 = parse_args(command2)
print(args2)

usage: colab_kernel_launcher.py [-h] [--save_steps SAVE_STEPS] [--save_as_full_pipeline]
                                [--num_vectors NUM_VECTORS] --pretrained_model_name_or_path
                                PRETRAINED_MODEL_NAME_OR_PATH [--revision REVISION]
                                [--variant VARIANT] [--tokenizer_name TOKENIZER_NAME]
                                --train_data_dir TRAIN_DATA_DIR --placeholder_token
                                PLACEHOLDER_TOKEN --initializer_token INITIALIZER_TOKEN
                                [--learnable_property LEARNABLE_PROPERTY] [--repeats REPEATS]
                                [--output_dir OUTPUT_DIR] [--seed SEED] [--resolution RESOLUTION]
                                [--center_crop] [--train_batch_size TRAIN_BATCH_SIZE]
                                [--num_train_epochs NUM_TRAIN_EPOCHS]
                                [--max_train_steps MAX_TRAIN_STEPS]
                                [--gradient_accumulation_step

SystemExit: 2

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
